In [15]:
# coding: utf-8
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

import os
current_dir = os.path.abspath(os.getcwd())
parent_dir = os.path.dirname(current_dir)

import sys
sys.path.append(current_dir)

import weth_api as wa
import gpt


In [2]:
import pytz
from timezonefinder import TimezoneFinder

In [3]:
# getting credles
forecast_settings = wa.read_yaml_config('config.yaml', section='api.weatherapi.com')
base_url = forecast_settings['url']
method = forecast_settings['method']
forecast_api_key = forecast_settings['api_key']

gpt_settings = wa.read_yaml_config('config.yaml', section='gpt')
gpt_api_key = gpt_settings['api_key']

In [35]:
# getting weahter forecast for current place for nearest day hourly
gwd = wa.get_weth_data(forecast_api_key, base_url, method, '36.757594, 31.45268', 7) # anatolia
# gwd = wa.get_weth_data(forecast_api_key, base_url, method, '55.752539, 37.808001', 7) # moscow
# gwd = wa.get_weth_data(forecast_api_key, base_url, method, '41.692665, 44.801543', 7) # tbilisi
# gwd = wa.get_weth_data(forecast_api_key, base_url, method, '40.714627, -74.002863', 7) # nyc
df = wa.load_weth_data_to_df(gwd)

In [36]:
df.sample(3)

,time,time_epoch,is_day,temp_c,feelslike_c,windchill_c,heatindex_c,condition,wind_kph,wind_dir,pressure_hg,precip_mm,humidity,cloud,will_it_rain,will_it_snow,chance_of_rain,chance_of_snow,uv,place,region,country,lat,lon
15,2024-07-27 15:00:00,1722081600,1,34.2,37.1,34.2,37.1,Sunny,11.9,ESE,753.061930,0.0,42,3,0,0,0,0,8.0,Manavgat,Antalya,Turkey,36.76,31.45
19,2024-07-26 19:00:00,1722009600,1,28.4,32.0,28.4,32.0,Sunny,6.8,SSW,750.811745,0.0,73,8,0,0,0,0,7.0,Manavgat,Antalya,Turkey,36.76,31.45
22,2024-07-28 22:00:00,1722193200,0,30.6,30.4,30.6,30.4,Clear,16.6,NNE,754.562053,0.0,36,0,0,0,0,0,0.0,Manavgat,Antalya,Turkey,36.76,31.45


In [37]:
# df[0:48].set_index('time')[['temp_c', 'feelslike_c']].plot()

In [38]:
import datetime 
from datetime import timedelta

now = datetime.datetime.now() 
now_round = now.replace(minute=0, second=0, microsecond=0)
forecast_finish = now_round + timedelta(hours=4) 

In [39]:
work_df = df[
    (df['time'] >= now_round) &
    (df['time'] < forecast_finish)
]
work_df['hour'] = work_df.index

/tmp/ipykernel_85692/1362429086.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  work_df['hour'] = work_df.index


In [40]:
def linear(x, k, b, c) -> float:
    """
    Просто прямая

    Параметры:
    x (int or float): переменная
    k,b,c (int or float): параметры функции

    Возвращает:
    значение функции y = f(x)
    """
    return k * x + b

In [41]:
metric_dct = {
                'temp_c': 'температура', 
                'feelslike_c': 'воспринимаемая температура',
                'windchill_c': 'температура ветра',
                'heatindex_c': 'воспринимаемый индекс',
                # 'condition': 'погода',
                'wind_kph': 'скорость ветра',
                # 'wind_dir': 'направление ветра',
                'pressure_hg': 'давление',
                'precip_mm': 'уровень осадков',
                'humidity': 'влажность',
                'cloud': 'облачность',
                # 'will_it_rain': 'будет ли дождь',
                # 'will_it_snow': 'будет ли снег',
                'chance_of_rain': 'вероятность дождя',
                'chance_of_snow': 'вероятность снега',
                'uv': 'УФ-индекс',
}

In [42]:
work_df[0:6]

,time,time_epoch,is_day,temp_c,feelslike_c,windchill_c,heatindex_c,condition,wind_kph,wind_dir,pressure_hg,precip_mm,humidity,cloud,will_it_rain,will_it_snow,chance_of_rain,chance_of_snow,uv,place,region,country,lat,lon,hour
22,2024-07-24 22:00:00,1721847600,0,27.3,30.3,27.3,30.3,Clear,3.6,ENE,750.061683,0.0,76,4,0,0,0,0,0.0,Manavgat,Antalya,Turkey,36.76,31.45,22
23,2024-07-24 23:00:00,1721851200,0,27.2,30.1,27.2,30.1,Clear,0.7,SE,750.061683,0.0,76,4,0,0,0,0,0.0,Manavgat,Antalya,Turkey,36.76,31.45,23
0,2024-07-25 00:00:00,1721854800,0,27.1,30.0,27.1,30.0,Clear,1.4,SE,750.061683,0.0,76,5,0,0,0,0,0.0,Manavgat,Antalya,Turkey,36.76,31.45,0
1,2024-07-25 01:00:00,1721858400,0,27.1,30.1,27.1,30.1,Clear,4.0,ESE,749.311621,0.0,77,5,0,0,0,0,0.0,Manavgat,Antalya,Turkey,36.76,31.45,1


In [43]:
def calc_trends(work_df, x, y):
    from scipy.optimize import curve_fit
    popt, pcov = curve_fit(linear, work_df[x], work_df[y], 
                        # bounds=[0, np.inf],
                        method='trf',  #'lm', 'trf', 'dogbox'
                        )
    res = ''
    if popt[0] >= 0.05:
        res = 1 
    elif popt[0] <= -0.05:
        res = -1
    else:
        res = 0
    return res
def calc_metric(work_df, x, y):
    min = np.min(work_df[y])
    max = np.max(work_df[y])
    mean = np.round(np.mean(work_df[y]),0)
    trend = calc_trends(work_df, x, y)

    txt = ''

    if trend == 1:
        txt = "В ближайшее время " + metric_dct[y] + " составит " + str(mean) + ": вырастет с " + str(min) + " до " + str(max) + '\n'
    elif trend == -1:
        txt = "В ближайшее время " + metric_dct[y] + " составит " + str(mean) + ": уменьшится с " + str(max) + " до " + str(min) + '\n'
    else:
        txt = "В ближайшее время " + metric_dct[y] + " составит " + str(mean) + " и практически не изменится"  + '\n'

    return txt

In [44]:
# TO DO: норма для времени и места

In [45]:
forec_txt = """
Дай, пожалуйста, пару рекомендаций как одеться семье по погоде.
Я пришлю тебе показатели прогноза, а ты кратко расскажешь, как лучше одеться:
сначала рекомендации для взрослых,
потом - рекомендации ребёнку 3-5 лет, если они отличаются от рекомендации для взрослых
Обобщать и писать про всякие закономерности не нужно, только рекомендации кратко и по делу, каждая рекомендация в отдельном абзаце
По прогнозу погоды в моей местности ожидаются следующие показатели: \n
"""
for metric in list(metric_dct.keys()):
    print(metric)
    forec_txt = forec_txt + calc_metric(work_df, 'hour', metric)

temp_c
feelslike_c
windchill_c
heatindex_c
wind_kph
pressure_hg
precip_mm
humidity
cloud
chance_of_rain
chance_of_snow
uv


In [46]:
with open('request.txt', 'w') as f:
    f.write(forec_txt)
    f.close()

In [47]:
# reading the request from the file
with open('request.txt', 'r') as f:
    message = f.read()

In [49]:
# sending requests to model and saving answer to file
answ = gpt.send_message(gpt_api_key, message, model='gpt-4o-mini')
with open('output.txt', 'w') as f:
    f.write(answ)
    f.close()